### **Generating all possible equations for [Mathler](https://mathler) game** *([Wordle](https://www.nytimes.com/games/wordle/index.html)'s mathematic variant)*

First of all, I thought this wouldn't be feasable, that the amount of possible expressions and results would be to large to compute, it turns out that both **Easy Mode** and **Medium Mode**, fit on 32GB of RAM *(even when converting `itertools.product(*pattern)` to a `list`)*.

But, **Hard Mode** didn't fit as a `list`, so I changed the approach to a 'lazy evaluation'. Where expressions and results would be written to the `.csv` file one at a time, instead of storing the whole dataset im RAM and then writing it to the `.csv`.

Let's list the explicit rules for each possible expression:
+ Each expression must be exactly 6 elements long *(each element is considered to be a single digit number or an operator)*
+ Numbers and operators can appear more than once
+ No leading zeros are allowed
+ Calculations follow the order of operations **PEDMAS**
    1. Parenthesis
    2. Exponents
    3. Division or Multiplication
    4. Addition or Subtraction

We can also extrapolate some other rules based on the previous requirements:

+ Expression can start with a `'+'` and `'-'` sign, but can't start with the other operators *(`'/','*'`)*
+ Expression can't end with any of the operators *(`'/','*','+','-'`)*
+ Only divisions that evaluate to an integer are valid
+ Divisions by zero are not allowed
+ There can't be expressions that are equal to the solution:
  + 6 digit numbers
  + 5 digit numbers that start with a `'+'` or a `'-'` sign

Besides all of that, there are 3 modes:

+ **Easy Mathler** *(5 squares and up to 1 operator)*
+ **Mathler** *(6 squares and up to 2 operators)*
+ **Hard Mathler** *(8 squares, parenthesis allowed and up to 3 operators)*

In [13]:
from tqdm import tqdm
import itertools
import csv
import re

from environment_helper import Environment

Defining constants:

*(Hard mode disabled for now, Mathler recently added the possibility for parenthesis in the expressions)*

In [14]:
FILE_PREFIX = "expressions"             # The prefix for the output files
CSV_HEADER = ["expression", "result"]   # The header for the CSV files

FIRST_ELEMENTS =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "+", "-"]              # First elements of the expression
MIDDLE_ELEMENTS = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "+", "-", "*", "/"]    # Middle elements of the expression
LAST_ELEMENTS =   ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]                        # Last elements of the expression

GAME_MODES = {  # Stores all the game modes data
    "easy": {
        "file_basename": f"{FILE_PREFIX}_easy",
        "max_operators": 1,
        "expressions_pattern": [FIRST_ELEMENTS] + [MIDDLE_ELEMENTS] * 3 + [LAST_ELEMENTS],
    },
    "medium": {
        "file_basename": f"{FILE_PREFIX}_medium",
        "max_operators": 2,
        "expressions_pattern": [FIRST_ELEMENTS] + [MIDDLE_ELEMENTS] * 4 + [LAST_ELEMENTS],
    },
    # "hard": {
    #     "file_basename": f"{FILE_PREFIX}_hard",
    #     "max_operators": 3,
    #     "expressions_pattern": [FIRST_ELEMENTS] + [MIDDLE_ELEMENTS] * 6 + [LAST_ELEMENTS],
    # },
}

Defining variables based on the current environment ("kaggle" or "local")

In [15]:
env = Environment(
    {
        "kaggle": {
            "split_files": False,
        },
        "local": {
            "split_files": True,
            "max_file_size": 99 * 1024 ** 2,
        },
    },
)

print(f"mode:\t\t{env.mode}")
print(f"dirpath:\t{env.dirpath}")

Function that solves an expression

In [16]:
def solve(expression: str):
    
    ##################################################
    # PRE VALIDATION
    ##################################################

    # There can't be consecutive "/" or "*" operators
    if re.findall(r"\D[\/,*]", expression):
        return None

    # Check for leading zeros
    if re.findall(r"0\d", expression):
        return None

    ##################################################
    # JOINING NUMBERS
    ##################################################

    # Convert expression to a list
    expression = list(expression)

    # Init temporary string and index number
    temp = ""
    i = 0

    # Loop through the expression
    while i < len(expression):

        # Check if the current element is a number
        if expression[i].isnumeric():

            # Add the number to the temporary string and pop it
            temp += expression[i]
            expression.pop(i)

        # Insert temp into expression if it's not empty
        else:
            if temp != "":
                expression.insert(i, temp)

            # Reset temp and go to the next element
            temp = ""
            i += 1

    # Append the last temp to the expression
    expression.append(temp)

    ##################################################
    # JOINING OPERATORS
    ##################################################

    # Init index
    i = 1

    # Loop through the whole expression
    while True:

        # Check if there are consecutive "+" and "-" operators
        if expression[i - 1] in "+-" and expression[i] in "+-":

            # If so, join the operators
            if expression[i - 1] == expression[i]:
                expression[i - 1] = "+"

            else:
                expression[i - 1] = "-"

            # Remove the current element and reset index
            expression.pop(i)
            i = 1

        # Go to next element if current ones are not "+" or "-" operators
        else:
            i += 1

        # Check if we've reached the end of the expression
        if i >= len(expression):
            break

    ##################################################
    # JOINING NUMBERS TO OPERATORS
    ##################################################

    # Init index
    i = 1

    # Loop through the expression while there are "+" and/or "-" operators
    while "+" in expression or "-" in expression:

        # Check if previous element is a "+" or "-" operator
        if expression[i - 1] in "+-":

            # If so, join the operator to the current number element, and pop current element
            expression[i - 1] += expression[i]
            expression.pop(i)

        # Go to next element
        i += 1

    ##################################################
    # CONVERTING NUMBERS TO INTS
    ##################################################

    # Loop through the expression
    for i in range(len(expression)):

        # Try to convert the element to an integer
        try:
            expression[i] = int(expression[i])
        except ValueError:
            pass

    ##################################################
    # CHECK FOR DIVISIONS BY 0
    ##################################################

    # Loop through the expression comparing one element at a time
    for i in range(1, len(expression)):

        # Return None if the previous element is a division operator and the current element is zero
        if expression[i - 1] == "/" and expression[i] == 0:
            return None

    ##################################################
    # SOLVING MULTIPLICATIONS AND DIVISIONS
    ##################################################

    # Init index
    i = 1

    # Loop through the expression while there are "/" and/or "*" operators
    while "/" in expression or "*" in expression:

        # Check if current element is a division or multiplication operator
        if expression[i] == "/" or expression[i] == "*":

            # Perform the appropriate operation
            if expression[i] == "/":
                expression[i - 1] /= expression[i + 1]

                # Return None if the division results is not an integer
                if not expression[i - 1] % 1 == 0:
                    return None

            else:
                expression[i - 1] *= expression[i + 1]

            # Remove the current element and the following element
            for _ in range(2):
                expression.pop(i)

        # Go to next element
        else:
            i += 1

    ##################################################
    # SUMMING ALL ELEMENTS
    ##################################################

    # Return sum only if expression is not empty
    if expression:
        return int(sum(expression))

    else:
        return None

Validating the Expression Solver function

In [17]:
# Define test cases
TEST_EQUATIONS = [
    ("-3+4/2", -1),
    ("-3*4/2", -6),
    ("-10+10", 0),
    ("10-1-9", 0),
    ("40/4/2", 5),
    ("1+2+30", 33),
    ("3+21/3", 10),
    ("40/4+2", 12),
    ("16+4*2", 24),
    ("2/1*40", 80),
    ("195-87", 108),
    ("1+2/30", None),
    ("2/3*10", None),
]

# Iterate over equations and check if they match the result
for expression, result_expected in TEST_EQUATIONS:
    result_solver = solve(expression)

    assert result_solver == result_expected, f"{expression} should be {result_expected} not {result_solver}"
else:
    print("All tests passed!")

Defining the function that generates all of the valid equations for a given game mode

In [18]:
def generate(mode: str):

    # Extract variables and store them
    output_file_basename = GAME_MODES[mode]['file_basename']
    max_operators = GAME_MODES[mode]["max_operators"]
    pattern = GAME_MODES[mode]["expressions_pattern"]

    # Define variables
    output_filepath = f"{env.dirpath}{output_file_basename}.csv"
    valid_expressions_counter = 0

    # Init output file
    with open(output_filepath, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(CSV_HEADER)

    # Calculate total number of candidate expressions
    candidates_count = 1
    for subpattern in pattern:
        candidates_count *= len(subpattern)

    # Create all combinations of elements for each position of the expressions pattern
    expressions = itertools.product(*pattern)

    # Iterate over each expression
    for expression in tqdm(expressions, total=candidates_count, desc=f"{mode} expressions\t"):

        # Convert expression to a single string
        expression = "".join(expression)

        # Check if operators count is less than max
        operators_count = 0

        for operator in "+-*/":
            operators_count += expression.count(operator)

        if operators_count > max_operators:
            continue

        # Solve the expression
        result = solve(expression)

        # Discard expressions with invalid results
        if result is None:
            continue

        # Discard expressions that are equal to the result
        try:
            if str(result) == str(int(expression)):
                continue
            
        except ValueError:
            pass

        # Append expression and result to CSV file
        with open(output_filepath, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([expression, result])
        
        # Increment valid expressions counter
        valid_expressions_counter += 1

    # Return the valid number of expressions generated
    return valid_expressions_counter


Generating all valid equations for each difficulty mode

In [20]:
generate("easy")

In [19]:
generate("medium")

In [ ]:
# generate("hard")